In [ ]:
%%writefile HelperFunctions/DataWrangling/criaCorpusWithOCR.py

# 1.Extração de texto dos pdf das sentenças baixas
#
# Cria Corpus (DatasetClassification.CSV) das Sentenças do TRT1
# Le os arquivos .txt e armazena-os em uma lista e depois persiste-o em disco no formato .csv.
#
import io
import pathlib
import pandas as pd
import datasets
from datasets import load_dataset
from datasets import Dataset
import os
import shutil
from os import listdir
from os.path import isfile, join
import re
# IMPORTS:
import cv2
import pytesseract
from PIL import Image, ImageOps
from pdf2image import convert_from_path
import numpy
    
class Corpus():
    
    def __init__(self):
        # Agrupar estas variaveis em um arquivo para carregar nas classes. VARIABLES:
        self.custom_config = r'--oem 1 --psm 4'
        self.pdfs_path = "/home/info/MyNotebooks/Datasets/SentencasTRT1/PDFs/"   #file_path = "/content/"
        self.file_path = "/home/info/MyNotebooks/Datasets/SentencasTRT1/PDFsProcessing/"
        self.output_folder="/home/info/MyNotebooks/Datasets/SentencasTRT1/IMGs/"  #arq = file_path+"00002067920105010076-29-03-2016.pdf"
        #self.arq = "0080300-68.2008.5.01.0016.pdf"
        self.arq = "" #"0012600-80.2005.5.01.0016.pdf"
        self.path_arq = "" #self.file_path+self.arq
        self.images = []
        self.textos = []
        self.pages = []

        
        
    def pdftoimg(self):
        
        self.pages = convert_from_path(self.path_arq, 500, output_folder=self.output_folder+"imgs/", fmt='jpg') # first_page=1, last_page=1, 

        for i, page in enumerate(self.pages):
            page_name = self.output_folder+self.arq+"_page_"+str(i)+".jpg"
            page.save(page_name, "JPEG")
            self.images.append(page_name)
        

        # Remove temporary imgs/*.jpg files:
        directoryList = os.listdir(self.output_folder+"imgs/")      
        for imgFile in directoryList:
            if imgFile.endswith('.jpg'):
                os.remove(self.output_folder+"imgs/"+imgFile)
            
    
    def crop_img(self):
        for i, page in enumerate(self.pages):
                        
            # open method used to open different extension image file
            # pil_image = Image.open(self.output_folder+self.arq+"_page_"+str(i)+".jpg") 
            #width, height = pil_image.size
            # gray_img    = ImageOps.grayscale(pil_image)
            # bk_wh_img   = gray_img.convert('1', dither=Image.NONE) # convert image to black and white
            
            img = cv2.imread(self.output_folder+self.arq+"_page_"+str(i)+".jpg", cv2.IMREAD_GRAYSCALE)
            # You may need to convert the color.
            #img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            (thresh, im_bw) = cv2.threshold(img, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
            #dst = cv2.detailEnhance(im_bw, sigma_s=10, sigma_r=0.15)
            
            pil_image = Image.fromarray(im_bw)

            frac   = 0.89
            left   = pil_image.size[0]*((1-frac)/2)
            upper  = pil_image.size[1]*((1-frac)/2)
            right  = pil_image.size[0]-((1-frac)/2)*pil_image.size[0]
            bottom = pil_image.size[1]-((1-frac)/2)*pil_image.size[1]
        
            cropped_img = pil_image.crop((left*1, upper*1, right, bottom*2.5))           
            cropped_img.save(self.output_folder+self.arq+"_page_"+str(i)+".jpg")      

         
    
    def imgtotext(self):
        
        self.textos = []
        
        for i, page in enumerate(self.pages):
            # Recognize the text as string in image using pytesserct 
            #pytesseract.pytesseract.tesseract_cmd = tesseract_path
            #text = str(((pytesseract.image_to_string(Image.open(img))))) 
            text = str(((pytesseract.image_to_string(self.output_folder+self.arq+"_page_"+str(i)+".jpg", lang='por', config=self.custom_config)))) 
            text.replace("\n", " ")
            text.replace('-\n', ' ')
            self.textos.append(text)

        
        imgList = os.listdir(self.output_folder)
        for k in imgList:
            if k.endswith('.jpg'):
                os.remove(self.output_folder+k)
                    
        print(len(self.images))
        res = " ".join(self.textos)
        return res

    
    
    def proccess_folder(self):
        
        txt_content = ""
        dataFile = {}
        directoryList = os.listdir(self.file_path)
        
        for pdfFile in directoryList:
            if pdfFile.endswith('.pdf'):
                self.arq = pdfFile
                self.path_arq = self.file_path+self.arq
                self.pages = []
                self.pdftoimg()
                self.crop_img()
                txt_content = self.imgtotext()  #
                with open(self.file_path+self.arq+'.txt', 'w') as f:
                    f.write(txt_content)
                f.close()
            txt_content = ""

                
                
    def reload_files(self):
        
        # 
        directoryList = os.listdir(self.pdfs_path)
        batch_step = 100
        for i in range(0,len(directoryList), batch_step):
            
            for j in range(i,i+batch_step):
                print(i,j,"moving for preparing proccess: ",directoryList[j])
                if directoryList[j].endswith('.pdf'):
                    shutil.move(self.pdfs_path+directoryList[j],self.file_path)
                    
            self.proccess_folder()
            
            for j in range(i,i+batch_step):
                print(i,j,"moving processed files: ",directoryList[j])
                shutil.move(self.file_path+directoryList[j],self.pdfs_path+"Processed/")
                shutil.move(self.file_path+directoryList[j]+".txt",self.file_path+"txts/")
                
            imgList = os.listdir(self.output_folder)
            for k in imgList:
                if k.endswith('.jpg'):
                    os.remove(self.output_folder+k)
            
        print("PDF files processed in: ", self.pdfs_path+"Processed/")
        print("TXT files generated in: ", self.file_path+"txts/")
        print("Temporary images files used in: ", self.output_folder)
        print("Eventually unprocessed/error files in: ", self.pdfs_path)

            
              
   

In [ ]:
# 2. Criação do objeto
from HelperFunctions.DataWrangling.criaCorpusWithOCR import Corpus
%run -i HelperFunctions/DataWrangling/criaCorpusWithOCR.py
corpus = Corpus()

In [ ]:
%%time
corpus.reload_files()

In [ ]:
# 7:51